## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-21-13-08-20 +0000,bbc,Nigeria police confirm mass church abductions ...,https://www.bbc.com/news/articles/c4grw9vg7y4o...
1,2026-01-21-13-04-00 +0000,wsj,Schwab’s Profit Jumps on Surge in Brokerage Ac...,https://www.wsj.com/finance/stocks/schwabs-pro...
2,2026-01-21-13-00-00 +0000,missionlocal,Longest-running Peruvian center in country los...,https://missionlocal.org/2026/01/sf-peruvian-c...
3,2026-01-21-12-55-15 +0000,nyt,‘Adaptive Normalcy’ as the Trump Show Arrives ...,https://www.nytimes.com/2026/01/21/business/de...
4,2026-01-21-12-52-40 +0000,nypost,Frigid Pittsburgh temperatures freeze over cit...,https://nypost.com/2026/01/21/us-news/pittsbur...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2371/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,88
97,greenland,38
28,davos,17
453,new,16
67,world,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
147,2026-01-21-03-26-41 +0000,nypost,Trump teases how far he’ll go to acquire Green...,https://nypost.com/2026/01/20/us-news/trump-te...,164
176,2026-01-21-01-05-00 +0000,wsj,President Trump’s campaign for Greenland follo...,https://www.wsj.com/politics/policy/trumps-gre...,164
123,2026-01-21-06-42-52 +0000,bbc,Trump doubles down on Greenland ahead of Davos...,https://www.bbc.com/news/articles/cvgr19m642zo...,160
244,2026-01-20-22-10-25 +0000,wapo,Stocks slide after Trump threatens new tariffs...,https://www.washingtonpost.com/business/2026/0...,159
360,2026-01-20-15-35-01 +0000,cbc,Stock markets plunge after Trump threatens som...,https://www.cbc.ca/news/business/wall-street-s...,159


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
147,164,2026-01-21-03-26-41 +0000,nypost,Trump teases how far he’ll go to acquire Green...,https://nypost.com/2026/01/20/us-news/trump-te...
173,63,2026-01-21-01-17-00 +0000,wsj,Escalating tensions over Greenland are superch...,https://www.wsj.com/economy/global/united-stat...
201,62,2026-01-20-23-45-18 +0000,latimes,State GOP seeks Supreme Court injunction to bl...,https://www.latimes.com/california/story/2026-...
137,56,2026-01-21-04-15-28 +0000,nypost,Air Force One forced to turn around after Trum...,https://nypost.com/2026/01/20/us-news/air-forc...
30,47,2026-01-21-11-51-26 +0000,nyt,Man Gets Life in Prison for Killing Shinzo Abe...,https://www.nytimes.com/2026/01/20/world/asia/...
68,43,2026-01-21-10-30-00 +0000,wsj,Google searches for “No Buy January” have hit ...,https://www.wsj.com/economy/consumers/no-buy-j...
330,43,2026-01-20-17-53-33 +0000,nypost,Iran warns that attack on Ayatollah ​Ali Khame...,https://nypost.com/2026/01/20/world-news/iran-...
242,43,2026-01-20-22-12-28 +0000,nyt,Minnesota Police Leaders Criticize Federal Tac...,https://www.nytimes.com/2026/01/20/us/ice-minn...
288,40,2026-01-20-20-19-00 +0000,wsj,Treasury Secretary Scott Bessent delivered his...,https://www.wsj.com/economy/central-banking/tr...
88,37,2026-01-21-09-55-53 +0000,bbc,Train driver killed in second deadly Spain rai...,https://www.bbc.com/news/articles/c1m78xl0gmpo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
